In [ ]:
!pip install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 16.0 MB/s eta 0:00:00


In [ ]:
!pip install langchain-openai
from langchain_core.runnables import RunnableLambda, RunnablePassthrough


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 639.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.2 MB/s eta 0:00:00


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [ ]:
from google.colab import userdata
import os
import pandas as pd
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
OPENAI = userdata.get('OPENAI_API_KEY')
# llm = ChatOpenAI(api_key=userdata.get('OPENAI_API_KEY'), model='gpt-3.5-turbo')
llm = ChatOpenAI(api_key=userdata.get('OPENAI_API_KEY'), model='gpt-4-turbo')
# llm = OpenAI(api_key=userdata.get('OPENAI_API_KEY'), model="gpt-4-turbo")
# df = pd.read_csv('training.csv')

In [ ]:
import csv
import json

In [ ]:
meaningful_prompt = PromptTemplate(
    input_variables=["text"],
    template="Is the following text meaningful? '{text}' Respond with Yes or No."
)

labeling_prompt =  """
Please analyze the following text to determine if it indicates a buying intention. Respond with a JSON object containing the following keys:

Text: {text}

buying_intention: 'Yes' or 'No' based on whether the text indicates a buying intention.
reason: The most important reason for the labeling.
text: The original input text.

*** Pay attention that the JSON format style is so crucial there shouldn't be any extra trailing comma or brackets in the JSON object.

Respond only with a JSON object.
For example:
{{
  "text": "@ siracusa  just listened to Hypercritical #16, whole episode about LCD ghosting and how horrible it is. Doesn't the OLED ghosting annoy you? It drives me crazy every time I scroll black backgrounds on an iPhone X",
  "buying_intention": "No",
  "reason": "The text discusses issues with current technology (OLED ghosting) and expresses dissatisfaction but does not indicate an intention to buy or look for a new product.",
}}

"""









LABELING_PROMPT = ChatPromptTemplate.from_template(labeling_prompt)



In [ ]:
# Process each row
import re
results = []
count = 0
for index, row in df.iterrows():
    # if count == 5: break
    text = row['text']
    old_label = row['class']
    prompt = labeling_prompt.format(text=text)
    # Skip rows without any text
    if pd.isna(text) or not text.strip():
        continue
    chain = LABELING_PROMPT | llm | StrOutputParser()


    value = chain.invoke({"text":text})

    # print(type(value))
    try:
      labeling = json.loads(value)
    except:
      try:
        # Clean up the JSON response by removing the trailing comma before the closing brace
        new_value = re.sub(r',\s*}', '}', response_content.strip())
        labeling = json.loads(new_value)
      except Exception as e:
        print("Error parsing JSON")
        print(e)
        print(value)
        continue
     # Access content of ChatOpenAI response
    labeling['old_label'] = old_label
    # print(labeling)
    results.append(labeling)
    print(count)
    count +=1

     # Append the parsed response

headers = ["text","old_label", "reason", "buying_intention"]

with open('results.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()
    writer.writerows(results)

print("Results have been written to 'results.csv'")

Error parsing JSON
name 'response_content' is not defined
{
  "text": "What's the difference between the  iPhone X  and the  iPhone X  X Max?",
  "buying_intention": "Yes",
  "reason": "The text is asking for a comparison between two products, typically indicating research prior to a potential purchase.",
}
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
Error parsing JSON
name 'response_content' is not defined
{
  "text": "# games_accessoriesLED  Digital USB Cable For  iPhone X  7 8 Plus 6 Charger 2.1A(Max) Voltage Current Display Charge Data Nylon Braided Cable  pic.twitter.com/aAyOjZwryd",
  "buying_intention": "No",
  "reason": "The text describes a product, likely in a promotional context, but does not explicitly indicate that the person posting has an intention to buy or is encouraging others to buy.",
}
33
34
35
36
Error parsing JSON
name 'response_content' is not defined
{
  "text": "protector, tempered, glass, safety, touch, high qualit

ValueError: dict contains fields not in fieldnames: 'buying_intition'

In [ ]:
prompt = """I need you to generate 10 different possible customer queries that are highly likely to be asked by customers with the intention of buying in the given category. Each query should include a "buy_intent" field indicating whether the intent is "Yes" or "No", and a "buy_prob" field indicating the probability of purchase with values ranging from 0 to 10, where 0 indicates no intent and 10 indicates the highest intent. The category is described below:

category: {category}

*** REMEMBER : DO NOT FORGET TO OUTPUT JUST A LIST OF JSON STRINGS.
*** REMEMBER : FOR THEY KEYS OF THE JSONS JUSE THE LOWERCASE VERSION OF THE KEY.
*** REMEMBER : JSON Expecting property name enclosed in double quotes
**** JUST USE THESE AS KEYS FOR JSON OBJECTS : "text", "buy_intent", "buy_prob", and "cat"

The returned response should be a list of JSON OBJECTS formatted as follows:
[
  {{
    "text": "@ EE  unhappy with service! Swapped from three to ee and have now had my  iPhone   X  delivery pushed back 3 times   bad  choice switching",
    "buy_intent": "No",
    "buy_prob": "0",
    "cat": category
  }},
  {{
    "text": "want to buy a laptop.",
    "buy_intent": "Yes",
    "buy_prob": "80",
    "cat":category
  }},
  ...
]
"""




In [ ]:
without_buy_intention_prompt = """I need you to generate 10 different possible customer queries that are highly likely to be asked by customers WITHOUT the intention of buying in the given category. Each query should include a "buy_intent" field indicating whether the intent is "Yes" or "No", and a "buy_prob" field indicating the probability of purchase with values ranging from 0 to 10, where 0 indicates no intent and 10 indicates the highest intent. The category is described below:

category: {category}

*** REMEMBER : DO NOT FORGET TO OUTPUT JUST A LIST OF JSON STRINGS.
*** REMEMBER : JUST GIVE ME SENTENCES WITHOUT INTENTION OF BUYING. (buy_intent:No)
*** REMEMBER : FOR THEY KEYS OF THE JSONS JUSE THE LOWERCASE VERSION OF THE KEY.
*** REMEMBER : JSON Expecting property name enclosed in double quotes
**** JUST USE THESE AS KEYS FOR JSON OBJECTS : "text", "buy_intent", "buy_prob", and "cat"

The returned response should be a list of JSON OBJECTS formatted as follows:
[
  {{
    "text": "@ EE  unhappy with service! Swapped from three to ee and have now had my  iPhone   X  delivery pushed back 3 times   bad  choice switching",
    "buy_intent": "No",
    "buy_prob": "0",
    "cat": category
  }},
  {{
    "text": "Dropped my 1 week review on the  iPhone   X ! My good and  bad  experiences shared. RTs Appreciated! Thanks!  https:// youtu.be/ed3r9bzzlgE Â  pic.twitter.com/aMa02K5njC",
    "buy_intent": "No",
    "buy_prob": "1",
    "cat":category
  }},
  ...
]
"""




In [ ]:
LABELING_PROMPT = ChatPromptTemplate.from_template(prompt)
chain = LABELING_PROMPT | llm | StrOutputParser()

categories_list = [
    "Home & Kitchen: This category remains immensely popular due to its broad appeal. Items such as kitchen gadgets, home decor, and storage solutions are consistently in demand (Jungle Scout) (Scale Insights).",
    "Beauty & Personal Care: With an ongoing focus on personal well-being, products in this category, including skincare, haircare, and cosmetics, continue to see high sales volumes (Jungle Scout) (SageMailer).",
    "Electronics: This category is driven by constant technological advancements and includes items like smartphones, home entertainment systems, and accessories. Consumers are always looking for the latest gadgets and devices (Seller Snap) (SageMailer).",
    "Clothing, Shoes & Jewelry: Fashion remains a significant driver of Amazon sales, with a vast array of products catering to different styles and preferences. This category includes everyday wear, luxury items, and everything in between (Jungle Scout).",
    "Toys & Games: Popular among children and adults alike, this category includes traditional toys, board games, and electronic games. It is especially strong during holiday seasons and gift-giving occasions (Seller Snap).",
    "Health, Household & Baby Care: This category covers essential products such as vitamins, cleaning supplies, and baby care items, which are necessary for everyday living and thus see consistent demand (Seller Snap) (Scale Insights).",
    "Sports & Outdoors: With a growing emphasis on fitness and outdoor activities, products like fitness equipment, camping gear, and sports apparel are highly sought after (Seller Snap) (SageMailer).",
    "Pet Supplies: As pet ownership increases, so does the demand for pet-related products. This category includes food, toys, and accessories for various pets (Seller Snap) (Scale Insights).",
    "Office Supplies: The rise of remote work has kept this category in high demand, with products like stationery, organizational tools, and office furniture being particularly popular (SageMailer) (Scale Insights).",
    "Automotive: This category, though less talked about, sees significant sales in products like car accessories, parts, and tools, catering to vehicle owners and enthusiasts (Jungle Scout)."
]


print(chain.invoke({"category": "Clothing, Shoes & Jewelry: Fashion remains a significant driver of Amazon sales, with a vast array of products catering to different styles and preferences. This category includes everyday wear, luxury items, and everything in between"}))

KeyboardInterrupt: 

In [ ]:
category_distribution_with_description = {
    "Home & Kitchen": {
        "description": "This category remains immensely popular due to its broad appeal. Items such as kitchen gadgets, home decor, and storage solutions are consistently in demand (Jungle Scout) (Scale Insights).",
        "percentage": 20
    },
    "Beauty & Personal Care": {
        "description": "With an ongoing focus on personal well-being, products in this category, including skincare, haircare, and cosmetics, continue to see high sales volumes (Jungle Scout) (SageMailer).",
        "percentage": 15
    },
    "Electronics": {
        "description": "This category is driven by constant technological advancements and includes items like smartphones, home entertainment systems, and accessories. Consumers are always looking for the latest gadgets and devices (Seller Snap) (SageMailer).",
        "percentage": 15
    },
    "Clothing, Shoes & Jewelry": {
        "description": "Fashion remains a significant driver of Amazon sales, with a vast array of products catering to different styles and preferences. This category includes everyday wear, luxury items, and everything in between (Jungle Scout).",
        "percentage": 10
    },
    "Toys & Games": {
        "description": "Popular among children and adults alike, this category includes traditional toys, board games, and electronic games. It is especially strong during holiday seasons and gift-giving occasions (Seller Snap).",
        "percentage": 10
    },
    "Health, Household & Baby Care": {
        "description": "This category covers essential products such as vitamins, cleaning supplies, and baby care items, which are necessary for everyday living and thus see consistent demand (Seller Snap) (Scale Insights).",
        "percentage": 10
    },
    "Sports & Outdoors": {
        "description": "With a growing emphasis on fitness and outdoor activities, products like fitness equipment, camping gear, and sports apparel are highly sought after (Seller Snap) (SageMailer).",
        "percentage": 8
    },
    "Pet Supplies": {
        "description": "As pet ownership increases, so does the demand for pet-related products. This category includes food, toys, and accessories for various pets (Seller Snap) (Scale Insights).",
        "percentage": 5
    },
    "Office Supplies": {
        "description": "The rise of remote work has kept this category in high demand, with products like stationery, organizational tools, and office furniture being particularly popular (SageMailer) (Scale Insights).",
        "percentage": 4
    },
    "Automotive": {
        "description": "This category, though less talked about, sees significant sales in products like car accessories, parts, and tools, catering to vehicle owners and enthusiasts (Jungle Scout).",
        "percentage": 3
    }
}


In [ ]:
lateral_question_prompt = """
I need you to generate 10 different possible customer queries that are initiation of a question that is more likely ended up purchasing and buying some stuff and highly likely to be asked by customers in the given category from Amazon Products. every example could be the beginning of a conversation, in the middle or evern in the end. Each text should include a "buy_intent" field indicating whether the intent is "Yes" or "No" Yes is showing that
the customer is highly likely gonna buy this stuff,  and a "buy_prob" field indicating the probability of purchase with values ranging from 0 to 10, where 0 indicates no intent and 10 indicates the highest intent. The category is described below:

category: {category}

*** REMEMBER : DO NOT GIVE ME A DIRECT INTENTION OF BUYING GIVE EXAMPLE THAT ARE INDIRECTLY MAY ENDED UP IN BUYING
    EXAMPLE: "Tell me which is the best smartphone on the market"
*** REMEMBER : DO NOT FORGET TO OUTPUT JUST A LIST OF JSON STRINGS.
*** REMEMBER : JUST GIVE ME SENTENCES WITHOUT INTENTION OF BUYING. (buy_intent:No)
*** REMEMBER : FOR THEY KEYS OF THE JSONS JUSE THE LOWERCASE VERSION OF THE KEY.
*** REMEMBER : JSON Expecting property name enclosed in double quotes
**** JUST USE THESE AS KEYS FOR JSON OBJECTS : "text", "buy_intent", "buy_prob", and "cat"

The returned response should be a list of JSON OBJECTS formatted as follows:
[
  {{
    "text": "Tell me which is the best smartphone on the market",
    "buy_intent": "Yes",
    "buy_prob": "6",
    "cat": category
  }},

  ...
]

"""

In [ ]:
total_number = 10
count = 0
error_count = 0
# Function to run the special command for the calculated number of times
def run_special_command(category, description, times):
    global count
    global error_count
    LABELING_PROMPT = ChatPromptTemplate.from_template(lateral_question_prompt)
    chain = LABELING_PROMPT | llm | StrOutputParser()
    for _ in range(times):
        # print(category)
        print(count)
        results = chain.invoke({"category": category+" "+description})
        # print(type(results))
        # Define the CSV file path

        #Convert each JSON-like string to a Python dictionary

        try:
            # Convert each JSON-like string to a Python dictionary
            parsed_json = json.loads(results)

        except json.JSONDecodeError as e:
          results = chain.invoke({"category": category+" "+description})
          try:
            # Convert each JSON-like string to a Python dictionary
            parsed_json = json.loads(results)

          except json.JSONDecodeError as e:
            print("Error parsing JSON")
            print(e)
            print(results)
            error_count += 1
            print(error_count)

            continue
        # Define the CSV file path
        csv_file_path = 'data.csv'

        # Check if the file exists
        file_exists = os.path.isfile(csv_file_path)

        # Open the CSV file in append mode, create if it does not exist
        with open(csv_file_path, mode='a', newline='') as file:
    # Define the field names (CSV columns)
          fieldnames = ["text", "buy_intent", "buy_prob", "cat"]

          # Create a DictWriter object
          writer = csv.DictWriter(file, fieldnames=fieldnames)

          # Write the header only if the file does not exist
          if not file_exists:
              writer.writeheader()

          # Write the parsed JSON objects to the CSV file
          for item in parsed_json:
              # if 'Category' in item:
              #     item['category'] = item.pop('Category')
              try:
                writer.writerow(item)
              except:
                print("Error writing to CSV")
                print(item)
                continue
        count += 1


# Iterate through the categories and calculate the number of times to run the special command
for category, details in category_distribution_with_description.items():
    times_to_run = (details['percentage'] * total_number)  // 100
    # if times_to_run == 0:
    #     times_to_run = 1;
    run_special_command(category, details['description'], times_to_run)

0
1
2
3
4
5
6


In [ ]:
final_prompt = """
Generate 20 diverse user messages that might be part of a conversation with an assistant or consultant about buying items. The messages should include different styles and stages of a conversation, from simple greetings to intermediate sentences, and direct expressions of buying intentions. Ensure the messages reflect a wide range of tones and authenticity to represent the majority of users. Cover the following categories of products:


{category}

Give me examples that the buying intention is "0" and there is

For each message, provide three features: "buy_inten" (1 if there is a buying intention, 0 if there isn't), "inten_level" (a number between 0 and 10 showing the strength of the buying intention), and "cat" (which is the category of the product; if the message does not directly fit any category above, put them in the 'General' category).

*** REMEMBER: DO NOT FORGET TO OUTPUT JUST A LIST OF JSON STRINGS.
*** REMEMBER: FOR THE KEYS OF THE JSONS JUST USE THE LOWERCASE OF THE KEYS, as follows: "text", "buy_inten", "inten_level", and "cat".
*** REMEMBER: JSON EXPECTS PROPERTY NAMES ENCLOSED IN DOUBLE QUOTES.
*** JUST USE THESE AS KEYS FOR JSON OBJECTS: "text", "buy_inten", "inten_level", and "cat"

# The returned response should be a list of JSON OBJECTS formatted as follows:


[
  {{
    "text": "Hello",
    "buy_inten": 0,
    "inten_level": 0,
    "cat": General
  }},
  {{
    "text": "How are you?",
    "buy_inten": 0,
    "inten_level": 0,
    "cat": General
  }},
    {{
      "text": "Can you suggest a good replacement on Amazon?",
      "buy_inten": 1,
      "inten_level": 7,
      "cat": General
    }},
    {{
      "text": "I'm looking for a new coffee maker",
      "buy_inten": 1,
      "inten_level": 8,
      "cat": Home & Kitchen
    }},
    {{
      "text": "Do you have any recommendations for skincare products?",
      "buy_inten": 1,
      "inten_level": 6,
      "cat": Beauty & Personal Care
    }},
    {{
      "text": "I need to buy a new phone charger",
      "buy_inten": 1,
      "inten_level": 9,
      "cat": Electronics
    }},
    {{
      "text": "Can you help me find a gift for my friend?",
      "buy_inten": 1,
      "inten_level": 5,
      "cat": General
    }},
    {{
      "text": "I'm thinking about getting a new laptop",
      "buy_inten": 1,
      "inten_level": 7,
      "cat": Electronics
    }},
    {{
      "text": "What are some good options for dog food?",
      "buy_inten": 1,
      "inten_level": 6,
      "cat": Pet Supplies
    }}
]
"""

# final_prompt = """
# Generate 20 diverse user messages that might be part of a conversation with an assistant or consultant about buying items from Amazon. The messages should include different styles and stages of a conversation, from simple greetings to intermediate sentences, and direct expressions of buying intentions. Ensure the messages reflect a wide range of tones and authenticity to represent the majority of users. Cover the following categories of products:

# {category}

# For each message, provide three features: "buy_inten" (1 if there is a buying intention, 0 if there isn't), "inten_level" (a number between 0 and 10 showing the strength of the buying intention), and "cat" (which is the category of the product; if the message does not directly fit any category above, put them in the 'General' category).

# *** REMEMBER: DO NOT FORGET TO OUTPUT JUST A LIST OF JSON STRINGS.
# *** REMEMBER: FOR THE KEYS OF THE JSONS JUST USE THE LOWERCASE OF THE KEYS, as follows: "text", "buy_inten", "inten_level", and "cat".
# *** REMEMBER: JSON EXPECTS PROPERTY NAMES ENCLOSED IN DOUBLE QUOTES.
# **** JUST USE THESE AS KEYS FOR JSON OBJECTS: "text", "buy_inten", "inten_level", and "cat"

# The returned response should be a list of JSON OBJECTS formatted as follows:

# [
#   {
#     "text": "Hello",
#     "buy_inten": 0,
#     "inten_level": 0,
#     "cat": "General"
#   },
#   {
#     "text": "How are you?",
#     "buy_inten": 0,
#     "inten_level": 0,
#     "cat": "General"
#   },
#   {
#     "text": "Can you suggest a good replacement on Amazon?",
#     "buy_inten": 1,
#     "inten_level": 7,
#     "cat": "General"
#   },
#   {
#     "text": "I'm looking for a new coffee maker",
#     "buy_inten": 1,
#     "inten_level": 8,
#     "cat": "Home & Kitchen"
#   },
#   {
#     "text": "Do you have any recommendations for skincare products?",
#     "buy_inten": 1,
#     "inten_level": 6,
#     "cat": "Beauty & Personal Care"
#   },
#   {
#     "text": "I need to buy a new phone charger",
#     "buy_inten": 1,
#     "inten_level": 9,
#     "cat": "Electronics"
#   },
#   {
#     "text": "Can you help me find a gift for my friend?",
#     "buy_inten": 1,
#     "inten_level": 5,
#     "cat": "General"
#   },
#   {
#     "text": "I'm thinking about getting a new laptop",
#     "buy_inten": 1,
#     "inten_level": 7,
#     "cat": "Electronics"
#   },
#   {
#     "text": "What are some good options for dog food?",
#     "buy_inten": 1,
#     "inten_level": 6,
#     "cat": "Pet Supplies"
#   }
# ]
# """


In [ ]:
zero_prompt = """
Generate 20 diverse user messages that might be part of a conversation with an assistant or consultant about buying items. The messages should include different styles and stages of a conversation, from simple greetings to intermediate sentences, and direct expressions of buying intentions. Ensure the messages reflect a wide range of tones and authenticity to represent the majority of users. Cover the following categories of products:


{category}

Give me examples that the buying intention is "0" and there is no intention of buying in the text. But, maybe there is some level of intention but no direct expression of buying intention in the text

For each message, provide three features: "buy_inten" (1 if there is a buying intention, 0 if there isn't), "inten_level" (a number between 0 and 10 showing the strength of the buying intention), and "cat" (which is the category of the product; if the message does not directly fit any category above, put them in the 'General' category).

*** REMEMBER: GIVE THE SAMPLE IN A WAY THAT THE INTENTION OF BUYING IS "0" AND THERE IS NO INTENTION OF BUYING IN THE TEXT
*** REMEMBER: DO NOT FORGET TO OUTPUT JUST A LIST OF JSON STRINGS.
*** REMEMBER: FOR THE KEYS OF THE JSONS JUST USE THE LOWERCASE OF THE KEYS, as follows: "text", "buy_inten", "inten_level", and "cat".
*** REMEMBER: JSON EXPECTS PROPERTY NAMES ENCLOSED IN DOUBLE QUOTES.
*** JUST USE THESE AS KEYS FOR JSON OBJECTS: "text", "buy_inten", "inten_level", and "cat"

# The returned response should be a list of JSON OBJECTS formatted as follows:


[
  {{
    "text": "Hello",
    "buy_inten": 0,
    "inten_level": 0,
    "cat": General
  }},
  {{
    "text": "How are you?",
    "buy_inten": 0,
    "inten_level": 0,
    "cat": General
  }},
    {{
      "text": "Can you suggest a good replacement on Amazon?",
      "buy_inten": 1,
      "inten_level": 7,
      "cat": General
    }},
    {{
      "text": "I'm looking for a new coffee maker",
      "buy_inten": 1,
      "inten_level": 8,
      "cat": Home & Kitchen
    }},
    {{
      "text": "Do you have any recommendations for skincare products?",
      "buy_inten": 1,
      "inten_level": 6,
      "cat": Beauty & Personal Care
    }},
    {{
      "text": "I need to buy a new phone charger",
      "buy_inten": 1,
      "inten_level": 9,
      "cat": Electronics
    }},
    {{
      "text": "Can you help me find a gift for my friend?",
      "buy_inten": 1,
      "inten_level": 5,
      "cat": General
    }},
    {{
      "text": "I'm thinking about getting a new laptop",
      "buy_inten": 1,
      "inten_level": 7,
      "cat": Electronics
    }},
    {{
      "text": "What are some good options for dog food?",
      "buy_inten": 1,
      "inten_level": 6,
      "cat": Pet Supplies
    }}
]
"""

In [ ]:
!pip install langchain-community langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.5 MB/s eta 0:00:00


In [ ]:
from langchain_community.callbacks import get_openai_callback

In [ ]:
import json


total_number = 100
count = 0
error_count = 0
import time

# Function to run the special command for the calculated number of times
def run_special_command(category, description, times):
    global count
    global error_count
    LABELING_PROMPT = ChatPromptTemplate.from_template(zero_prompt)
    chain = LABELING_PROMPT | llm | StrOutputParser()
    for _ in range(times):
        # print(category)
        print(count)
        # results = chain.invoke({"category": category+" "+description})

        with get_openai_callback() as cb:
            results = chain.invoke({"category": category+" "+description})
            print(cb)

        # print(type(results))
        # Define the CSV file path

        #Convert each JSON-like string to a Python dictionary

        try:
            # Convert each JSON-like string to a Python dictionary
            parsed_json = json.loads(results)

        except json.JSONDecodeError as e:
          results = chain.invoke({"category": category+" "+description})
          try:
            # Convert each JSON-like string to a Python dictionary
            parsed_json = json.loads(results)

          except json.JSONDecodeError as e:
            print("Error parsing JSON")
            print(e)
            print(results)
            error_count += 1
            print(error_count)

            continue
        # Define the CSV file path
        csv_file_path = 'dataset.csv'

        # Check if the file exists
        file_exists = os.path.isfile(csv_file_path)

        # Open the CSV file in append mode, create if it does not exist
        with open(csv_file_path, mode='a', newline='') as file:
    # Define the field names (CSV columns)
          fieldnames = ["text", "buy_inten", "inten_level", "cat"]

          # Create a DictWriter object
          writer = csv.DictWriter(file, fieldnames=fieldnames)

          # Write the header only if the file does not exist
          if not file_exists:
              writer.writeheader()

          # Write the parsed JSON objects to the CSV file
          for item in parsed_json:
              # if 'Category' in item:
              #     item['category'] = item.pop('Category')
              try:
                writer.writerow(item)
              except:
                print("Error writing to CSV")
                print(item)
                continue
        count += 1
        time.sleep(10)


# Iterate through the categories and calculate the number of times to run the special command
for category, details in category_distribution_with_description.items():
    times_to_run = (details['percentage'] * total_number)  // 100
    # if times_to_run == 0:
    #     times_to_run = 1;
    run_special_command(category, details['description'], times_to_run)

0
Tokens Used: 1704
	Prompt Tokens: 779
	Completion Tokens: 925
Successful Requests: 1
Total Cost (USD): $0.03554
Error writing to CSV
{'text': 'Do you have any tips on choosing curtains?', 'buy_inten': 0, 'inten by_level': 2, 'cat': 'Home & Kitchen'}
1
Tokens Used: 1719
	Prompt Tokens: 779
	Completion Tokens: 940
Successful Requests: 1
Total Cost (USD): $0.035989999999999994
2
Tokens Used: 1698
	Prompt Tokens: 779
	Completion Tokens: 919
Successful Requests: 1
Total Cost (USD): $0.03536
3
Tokens Used: 1696
	Prompt Tokens: 779
	Completion Tokens: 917
Successful Requests: 1
Total Cost (USD): $0.0353
Error parsing JSON
Expecting property name enclosed in double quotes: line 69 column 1 (char 1609)
[
  {
    "text": "Hello there!",
    "buy_inten": 0,
    "inten_level": 0,
    "cat": "General"
  },
  {
    "text": "What's the latest trend in home decor?",
    "buy_inten": 0,
    "inten_level": 0,
    "cat": "Home & Kitchen"
  },
  {
    "text": "Could you explain the benefits of silicone 

In [ ]:
import pandas as pd

# Load your CSV dataset
df = pd.read_csv('dataset_wop.csv')

# Display the original number of rows
print(f'Original number of rows: {df.shape[0]}')

# Remove duplicate texts
df_cleaned = df.drop_duplicates(subset='text')

# Display the number of rows after removing duplicates
print(f'Number of rows after removing duplicates: {df_cleaned.shape[0]}')

# Save the cleaned dataset
df_cleaned.to_csv('cleaned_dataset.csv', index=False)

print('Removed duplicates and saved the cleaned dataset.')

Original number of rows: 11983
Number of rows after removing duplicates: 11983
Removed duplicates and saved the cleaned dataset.


In [ ]:
import string
import nltk
from nltk.corpus import stopwords

In [ ]:
import ace_tools as tools;

ModuleNotFoundError: No module named 'ace_tools'

In [ ]:
df = pd.read_csv('dataset_wop.csv')

# Step 1: Review Data Quality
# Check for missing values
missing_values = df.isnull().sum()

# Step 2: Normalize Text
def normalize_text(text):
    text = text.lower()  # Convert to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    return text

df['text'] = df['text'].apply(normalize_text)

# Step 3: Tokenization and Removing Stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

df['text'] = df['text'].apply(remove_stopwords)

# Step 4: Label Encoding
from sklearn.preprocessing import LabelEncoder

# Encode categorical column 'cat'
le = LabelEncoder()
df['cat'] = le.fit_transform(df['cat'])


# Save cleaned dataset
cleaned_file_path = 'wop_dataset.csv'
df.to_csv(cleaned_file_path, index=False)

missing_values, df.head(), cleaned_file_path

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(text           0
 buy_inten      0
 inten_level    0
 cat            0
 dtype: int64,
                                                 text  buy_inten  inten_level  \
 0                              hello hope youre well          0            0   
 1  im looking stylish yet affordable home decor i...          1            7   
 2  recommend functional storage solutions small a...          1            8   
 3                need new set knives kitchen suggest          1            9   
 4            whats best selling kitchen gadget right          1            6   
 
    cat  
 0   14  
 1   34  
 2   34  
 3   34  
 4   34  ,
 'wop_dataset.csv')

In [ ]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder

# Load dataset
file_path = '/mnt/data/cleaned_dataset.csv'
df = pd.read_csv(file_path)

# Step 1: Review Data Quality
# Check for missing values
missing_values = df.isnull().sum()

# Step 2: Normalize Text
def normalize_text(text):
    text = text.lower()  # Convert to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    return text

df['text'] = df['text'].apply(normalize_text)

# Step 3: Tokenization and Removing Stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

df['text'] = df['text'].apply(remove_stopwords)

# Step 4: Label Encoding
le = LabelEncoder()
df['cat_encoded'] = le.fit_transform(df['cat'])

# Create a mapping of categories to their encoded values
category_mapping = dict(zip(le.classes_, le.transform(le.classes_)))


# Save cleaned dataset and mapping
cleaned_file_path = 'cleaned_dataset_processed.csv'
mapping_file_path = 'category_mapping.csv'
df.to_csv(cleaned_file_path, index=False)

# Save the category mapping
mapping_df = pd.DataFrame(list(category_mapping.items()), columns=['Category', 'Encoded Value'])
mapping_df.to_csv(mapping_file_path, index=False)

missing_values, df.head(), cleaned_file_path, mapping_file_path

In [ ]:
import pandas as pd

# Load dataset
file_path = 'dataset_wop.csv'
df = pd.read_csv(file_path)

# Get unique values of the 'cat' column
unique_categories = df['cat'].unique()

unique_categories

array(['General', 'Home & Kitchen', 'Beauty & Personal Care',
       'Electronics', 'Clothing, Shoes & Jewelry', 'Toys & Games',
       'Health, Household & Baby Care', 'Sports & Outdoors',
       'Pet Supplies', 'Office Supplies', 'Automotive'], dtype=object)

In [ ]:
import pandas as pd

# Load your CSV dataset
df = pd.read_csv('dataset_wop.csv')

# Display the original number of rows
print(f'Original number of rows: {df.shape[0]}')

# Remove duplicate texts, keeping the first occurrence
df_cleaned = df.drop_duplicates(subset='text', keep='first')

# Display the number of rows after removing duplicates
print(f'Number of rows after removing duplicates: {df_cleaned.shape[0]}')

# Save the cleaned dataset
df_cleaned.to_csv('cleaned_dataset.csv', index=False)

print('Removed duplicates and saved the cleaned dataset.')

Original number of rows: 11983
Number of rows after removing duplicates: 11983
Removed duplicates and saved the cleaned dataset.
